In [34]:
from src import DATA_DIR
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [69]:
def Reshape_Dataset(data_path, output_dir_path, mode=1):
    mode = 1

    df = pd.read_csv(data_path)
    df = df.astype({'Date': 'datetime64[ns]'})

    if mode == 1:
        df['Score HomeTeam'] = 0
        df['Score AwayTeam'] = 0

        df.loc[df.FTHG > df.FTAG, 'Score HomeTeam'] = 3
        df.loc[df.FTHG < df.FTAG, 'Score AwayTeam'] = 3
        df.loc[df.FTHG == df.FTAG, ['Score HomeTeam', 'Score AwayTeam']] = 1    

        list_ = []
        for i in range(len(df)):
            if df.iloc[i, 5] > 0:
                list_.append((df.iloc[i, 0], df.iloc[i, 1], df.iloc[i, 5]))

            if df.iloc[i, 6] > 0:
                list_.append((df.iloc[i, 0], df.iloc[i, 2], df.iloc[i, 6]))

        df = pd.DataFrame(list_, columns=['Date', 'Team', 'Score'])


    final_df = df.pivot(index='Date', columns='Team', values='Score')
    final_df = final_df.fillna(0)

    # for j, col in enumerate(final_df.columns):
    #     final_df.iloc[0, j] += int(df_Start_Score.loc[df_Start_Score.Team == col, 'Pt.'])

    for i in range(1, len(final_df)):
        final_df.iloc[i, :] += final_df.iloc[i - 1, :]


    dates = set([(date.year, date.month) for date in final_df.index])
    dates.add(((max(final_df.index) + relativedelta(months=1)).year, (max(final_df.index) + relativedelta(months=1)).month))
    dates = sorted(set(dates))

    df_ = []
    for i in range(len(dates)-1):
        df = final_df.loc[(final_df.index < datetime.datetime(dates[i+1][0], dates[i+1][1], 1, 0, 0)) &
                          (final_df.index >= datetime.datetime(dates[i][0], dates[i][1], 1, 0, 0))]

        # df['Date'] = datetime.datetime(dates[i+1][0], dates[i+1][1], 1, 0, 0)

        if not len(df):
            continue

        if not len(df_):
            df_ = df.loc[df.index == max(df.index)]

        else:
            df_ = pd.concat([df_, df.loc[df.index == max(df.index)]])


    df_['Date'] = None

    for i in range(len(df_)):
        df_.iloc[i, len(df_.columns)-1] = datetime.datetime(df_.index[i].year, df_.index[i].month, 1, 0, 0) + relativedelta(months=1)

    df_ = df_.set_index('Date')
    df_.to_csv(output_dir_path)

Your dataset can have one of the following formats:

* mode = 1
|    | Date      | HomeTeam    | AwayTeam   |   FTHG |   FTAG |
|---:|:----------|:------------|:-----------|-------:|-------:|
|  0 | 8/14/1993 | Arsenal     | Coventry   |      0 |      3 |
|  1 | 8/14/1993 | Aston Villa | QPR        |      4 |      1 |
|  2 | 8/14/1993 | Chelsea     | Blackburn  |      1 |      2 |


* mode = 2
|    | Date      | Team        |   Score |
|---:|:----------|:------------|--------:|
|  0 | 8/14/1993 | Arsenal     |       0 |
|  1 | 8/14/1993 | Aston Villa |       3 |
|  2 | 8/14/1993 | Chelsea     |       1 |

In [71]:
Reshape_Dataset(
    data_path = DATA_DIR / 'data/00_Sample_Format/Sample_One.csv',
    output_dir_path = DATA_DIR / 'data/02_Reshaped_Data/Dataset.csv',
    mode=1)